In [17]:
from src.json_models.src.model_generator import ModelGenerator

In [18]:
gen = ModelGenerator(json_path="/home/andrewheschl/PycharmProjects/classification_pipeline/models/mobilenets/mobilenet_xmodule.json")

In [19]:
model = gen.get_model()

In [20]:
import torch
data = torch.ones((1, 3, 320, 320))
out = model(data)
out.shape

torch.Size([1, 7])

In [27]:
out

tensor([[-0.0266,  0.0634, -0.0268,  0.1613,  0.0394, -0.0332,  0.0438]],
       grad_fn=<AddmmBackward0>)